In [1]:
%matplotlib inline

from config import configs

import os
import io
import numpy as np
import matplotlib
import timeit
import random
import cv2
import deepdish as dd

import matplotlib.pyplot as plt

from pprint import pprint
from datetime import datetime
from pixor_targets import PIXORTargets
from skimage.transform import resize
from scipy.interpolate import griddata
from core.kitti import KITTI
from pixor_utils.model_utils import load_model, save_model
from data_utils.training_gen import TrainingGenerator
from data_utils.generator import Generator, KITTIGen, KITTICarRecognition2DGen
from tt import bev
from data_utils.generator import KITTIGen, KITTIValGen
from pixor_utils.post_processing import nms_bev
from test_utils.unittest import test_pc_encoder, test_target_encoder
from encoding_utils.pointcloud_encoder import OccupancyCuboidKITTI
from encoding_utils.voxelizer import BEVVoxelizer

In [2]:
DS_DIR = os.path.expanduser(configs['dataset_path'])

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [3]:
# Point Cloud Encoder
INPUT_SHAPE = configs['input_shape']

# Training
BATCH_SIZE = configs['hyperparams']['batch_size']
LEARNING_RATE = configs['hyperparams']['lr']
EPOCHS = configs['hyperparams']['epochs']
NUM_THREADS = configs['hyperparams']['num_threads']
MAX_Q_SIZE = configs['hyperparams']['max_q_size']

In [4]:
kitti = KITTI(DS_DIR, configs['training_target'])

train_ids = kitti.get_ids('train')
val_ids = kitti.get_ids('val')
micro_ids = kitti.get_ids('micro')

In [5]:
print(len(train_ids) // 2)

1856


In [6]:
pc_encoder = OccupancyCuboidKITTI(0, 70, -40, 40, -1, 3, [0.1, 0.1, 0.4]) # 0.1, 0.1, 0.4
pc_encoder = BEVVoxelizer(n_x=448, n_y=512, n_z=32, side_range=(0,70), fwd_range=(-40,40), height_range=(-1, 3), filter_external=True)

In [7]:
target_encoder = PIXORTargets(shape=(128, 112), 
                              stats=dd.io.load('kitti_stats/new_stats.h5'),
                              P_WIDTH=70, P_HEIGHT=80, P_DEPTH=4, 
                              subsampling_factor=(0.75, 1.25))

In [8]:
pts, ref = kitti.get_velo(train_ids[25], workspace_lim=((-35, 35), (-1, 3), (0, 70)), use_fov_filter=True)  # Load velo
_, _, P2 = kitti.get_calib(train_ids[25])

a_s = timeit.default_timer()
start = timeit.default_timer()
depth_map = kitti.get_range_view(img=None, pts=pts, ref=ref, P2=P2, gt_boxes=None, pred_boxes=None, out_type='depth')
print('depth_map in:', timeit.default_timer() - start)

start = timeit.default_timer()
intensity_map = kitti.get_range_view(img=None, pts=pts, ref=ref, P2=P2, gt_boxes=None, pred_boxes=None, out_type='intensity')
print('intensity_map in:', timeit.default_timer() - start)

start = timeit.default_timer()
height_map = kitti.get_range_view(img=None, pts=pts, ref=ref, P2=P2, gt_boxes=None, pred_boxes=None, out_type='height')
print('height_map in:', timeit.default_timer() - start)

print('all took', timeit.default_timer() - a_s)

depth_map in: 0.07070790976285934
intensity_map in: 0.06788025330752134
height_map in: 0.07768896594643593
all took 0.21733012422919273


In [9]:
voxelizer  = BEVVoxelizer(n_x=448, n_y=512, n_z=32, side_range=(0,70), fwd_range=(-40,40), height_range=(-1, 3), filter_external=True)
a_s = timeit.default_timer()
voxelized_pc = voxelizer.encode(pts)
print('took', timeit.default_timer() - a_s)

took 0.1371385669335723


In [10]:
# a_s = timeit.default_timer()
# img, dmap, imap, hmap = kitti.get_rangeview_preprocessing(train_ids[25])
# mapping_2x, mapping_4x, mapping_8x, geo_2x, geo_4x, geo_8x = kitti.get_contfuse_preprocessing(train_ids[25])
# print('all took', timeit.default_timer() - a_s)

In [11]:
# train_gen = KITTICarRecognition2DGen(kitti, train_ids, 1)
# for batch in train_gen:
#     rgb_img, depth_map, intensity_map, height_map, target_map_high, target_map_mid, target_map_low = batch
#     print('rgb_img.shape          ', rgb_img.shape)
#     print('depth_map.shape        ', depth_map.shape)
#     print('intensity_map.shape    ', intensity_map.shape)
#     print('height_map.shape       ', height_map.shape)
#     print('target_map_high        ', target_map_high.shape)
#     print('target_map_mid         ', target_map_mid.shape)
#     print('target_map_low         ', target_map_low.shape)

In [12]:
train_gen = KITTIGen(kitti, train_ids, BATCH_SIZE, pc_encoder=pc_encoder, target_encoder=target_encoder, aug=True)
for batch in train_gen:
# #     start = timeit.default_timer()
    bev, rgb_img, depth_map, intensity_map, height_map, m2x, m4x, m8x, g2x, g4x, g8x, (obj3d, geo), ids = batch
# #     print('took {}'.format(timeit.default_timer() - start))
    print('bev.shape            ', bev.shape)
    print('rgb_img.shape        ', rgb_img.shape)
    print('depth_map.shape      ', depth_map.shape)
    print('intensity_map.shape  ', intensity_map.shape)
    print('height_map.shape     ', height_map.shape)
#     print('obj2d.shape          ', obj2d.shape)
    print('obj3d.shape          ', obj3d.shape)
    print('geo.shape            ', geo.shape)
    print('m2x.shape            ', m2x.shape)
    print('m4x.shape            ', m4x.shape)
    print('m8x.shape            ', m8x.shape)
    print('g2x.shape            ', g2x.shape)
    print('g4x.shape            ', g4x.shape)
    print('g8x.shape            ', g8x.shape)
    print('------------------------')

[4.869722260092491, -0.4284649141206981, 0.0]
0.936511957388956
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape             (2, 112, 128, 2)
m8x.shape             (2, 56, 64, 2)
g2x.shape             (2, 224, 256, 3)
g4x.shape             (2, 112, 128, 3)
g8x.shape             (2, 56, 64, 3)
------------------------
[-1.8241063287837789, 0.331716234438429, 0.0]
0.9866430171296339
[2.6141822393771976, -0.29045403793306823, 0.0]
1.0912240244606832
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 1

bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape             (2, 112, 128, 2)
m8x.shape             (2, 56, 64, 2)
g2x.shape             (2, 224, 256, 3)
g4x.shape             (2, 112, 128, 3)
g8x.shape             (2, 56, 64, 3)
------------------------
[4.60617679967908, -0.12293074922194958, 0.0]
1.014644384003204
[-2.0581313945101765, 0.24567290756629268, 0.0]
1.0212591444878807
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape          

[0.40671981471117746, 0.13526612679129368, 0.0]
1.0459240224636632
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape             (2, 112, 128, 2)
m8x.shape             (2, 56, 64, 2)
g2x.shape             (2, 224, 256, 3)
g4x.shape             (2, 112, 128, 3)
g8x.shape             (2, 56, 64, 3)
------------------------
[-3.218709418670622, 0.2557262271479792, 0.0]
0.9018273034580666
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape         

[-4.3033879792379555, -0.32383907955854674, 0.0]
1.0131249852712911
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape             (2, 112, 128, 2)
m8x.shape             (2, 56, 64, 2)
g2x.shape             (2, 224, 256, 3)
g4x.shape             (2, 112, 128, 3)
g8x.shape             (2, 56, 64, 3)
------------------------
[3.341758259291911, 0.4745940133558716, 0.0]
0.994411386463527
[-0.3762195917553388, -0.1033617540361561, 0.0]
0.9515263168185131
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112,

bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape             (2, 112, 128, 2)
m8x.shape             (2, 56, 64, 2)
g2x.shape             (2, 224, 256, 3)
g4x.shape             (2, 112, 128, 3)
g8x.shape             (2, 56, 64, 3)
------------------------
[-4.512120180214529, 0.3296077990393863, 0.0]
1.0347508815983242
[-3.996118728816879, 0.39147677798073754, 0.0]
1.0042197808097686
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape          

[-1.5058036435366065, -0.22165075433978454, 0.0]
1.0326334412230813
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2, 112, 128, 9)
m2x.shape             (2, 224, 256, 2)
m4x.shape             (2, 112, 128, 2)
m8x.shape             (2, 56, 64, 2)
g2x.shape             (2, 224, 256, 3)
g4x.shape             (2, 112, 128, 3)
g8x.shape             (2, 56, 64, 3)
------------------------
[-2.6933072213534635, -0.06870488518592788, 0.0]
0.9337173323475367
[-1.0291319875729412, -0.3955179814537332, 0.0]
0.9299587058271378
bev.shape             (2, 448, 512, 32)
rgb_img.shape         (2, 375, 1242, 3)
depth_map.shape       (2, 375, 1242, 1)
intensity_map.shape   (2, 375, 1242, 1)
height_map.shape      (2, 375, 1242, 1)
obj3d.shape           (2, 112, 128, 2)
geo.shape             (2,

KeyboardInterrupt: 

In [ ]:
# cur_idx = train_ids[159]

In [ ]:
# boxes = kitti.get_boxes_3D(cur_idx)
# encoded = target_encoder.encode(boxes)

In [ ]:
# outmap = np.squeeze(np.concatenate((encoded[0][:,:,0:1], encoded[1]), axis=-1))
# outmap.shape

In [ ]:
# org_boxes = target_encoder.decode(outmap, 0.8)
# nms  = nms_bev('dist', -2.5, max_boxes=20)
# org_boxes = nms(org_boxes)
# print(len(boxes), len(org_boxes))

In [ ]:
# # plt.figure(figsize = (5,5))
# # plt.imshow(np.squeeze(encoded[0][:,:,0]), cmap='gray')

# plt.figure(figsize = (5,5))
# plt.imshow(np.squeeze(encoded[0][:,:,1]), cmap='gray')

In [ ]:
# org_encoded = target_encoder.encode(org_boxes)
# # plt.figure(figsize = (5,5))
# # plt.imshow(np.squeeze(org_encoded[0][:,:,0]), cmap='gray')

# plt.figure(figsize = (5,5))
# plt.imshow(np.squeeze(org_encoded[0][:,:,1]), cmap='gray')

In [ ]:
# cos, sin = [], []
# for id in train_ids:
#     boxes = kitti.get_boxes_3D(id)
#     for box in boxes:
#         cur_yaw = box.yaw
#         if cur_yaw < 0:
#             cur_yaw *= -1
#         cos.append(np.cos(cur_yaw))
#         sin.append(np.sin(cur_yaw))
# print('cos_mean', np.mean(cos), 'cos_std', np.std(cos))
# print('sin_mean', np.mean(sin), 'sin_std', np.std(sin))